<a href="https://colab.research.google.com/github/leegyuhi/oooooooooooooh/blob/master/FEniCS_Ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch.2 Fundamentals: Solving the Poisson equation
## 2.2 FEniCS implementation
- 2D Poisson eqauation 예제
>$-\bigtriangledown ^{2}u=f,\: \: \: \: \: \: \: \: \: x\: in\: \Omega$
>
>$\: \: \: \: \: \: \: \: \: \: \: \: u=u_{D},\: \: \: \: \: \: x\: on\: \partial \Omega $
>
>$u_{D}(x,y)=1+x^{2}+2y^{2}$
>
>$f=-6$


In [0]:
from __future__ import print_function
from fenics import *
import matplotlib.pyplot as plt

# Create mesh and define function space
mesh = UnitSquareMesh(8, 8)
V = FunctionSpace(mesh, 'P', 1)

# Define boundary condition
u_D = Expression('1 + x[0]*x[0] + 2*x[1]*x[1]', degree=2)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u_D, boundary)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(-6.0)
a = dot(grad(u), grad(v))*dx
L = f*v*dx

# Compute solution
u = Function(V)
solve(a == L, u, bc)

# Plot solution and mesh
plot(u)
plot(mesh)

# Save solution to file in VTK format
vtkfile = File('poisson/solution.pvd')
vtkfile << u

# Compute error in L2 norm
error_L2 = errornorm(u_D, u, 'L2')

# Compute maximum error at vertices
vertex_values_u_D = u_D.compute_vertex_values(mesh)
vertex_values_u = u.compute_vertex_values(mesh)
import numpy as np
error_max = np.max(np.abs(vertex_values_u_D - vertex_values_u))

# Print errors
print('error_L2  =', error_L2)
print('error_max =', error_max)

# Hold plot
plt.show()

error_L2 = 0.00823509807335483

error_max = 1.3322676295501878e-15

![test problem](https://user-images.githubusercontent.com/52767505/74707768-1a4ed980-525e-11ea-9f93-eddae1d670dd.GIF)
# ![test problem_2](https://user-images.githubusercontent.com/52767505/74707770-1c189d00-525e-11ea-8b1d-a4ee5a2dbf10.GIF)

- Paraview를 이용한 솔루션 시각화
```
Error reading cell offsets: Unsupported array type: vtkUnsingnedlntArray
```


- Computing the error
  - error_L2 = 0.00823509807335483 ($L^{2}$ norm 에러)
  > $$E=\sqrt{\int_{\Omega } (u_{D}-u)^{2}dx}$$
  - error_max = 1.3322676295501878e-15
  > infinite element mesh의 모든 정점에서 오류의 최댓값 계산

## 2.4 Deflection of a membrane
$$-\bigtriangledown ^{2}w=4\, exp\left ( -\beta ^{2}\left ( x ^{2}+(y-R_{0}) ^{2} \right )  \right )$$

$$D=\frac{AR^{2}}{8\pi \sigma T}w$$

In [0]:
from __future__ import print_function
from fenics import *
from mshr import *
import numpy as np

# Create mesh and define function space
domain = Circle(Point(0, 0), 1)
mesh = generate_mesh(domain, 64)
V = FunctionSpace(mesh, 'P', 2)

# Define boundary condition
w_D = Constant(0)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, w_D, boundary)

# Define load
beta = 8
R0 = 0.6
p = Expression('4*exp(-pow(beta, 2)*(pow(x[0], 2) + pow(x[1] - R0, 2)))',
               degree=1, beta=beta, R0=R0)

# Define variational problem
w = TrialFunction(V)
v = TestFunction(V)
a = dot(grad(w), grad(v))*dx
L = p*v*dx

# Compute solution
w = Function(V)
solve(a == L, w, bc)

# Plot solution
p = interpolate(p, V)
plot(w, title='Deflection')
plot(p, title='Load')

# Save solution to file in VTK format
vtkfile_w = File('poisson_membrane/deflection.pvd')
vtkfile_w << w
vtkfile_p = File('poisson_membrane/load.pvd')
vtkfile_p << p

# Curve plot along x = 0 comparing p and w
import numpy as np
import matplotlib.pyplot as plt
tol = 0.001  # avoid hitting points outside the domain
y = np.linspace(-1 + tol, 1 - tol, 101)
points = [(0, y_) for y_ in y]  # 2D points
w_line = np.array([w(point) for point in points])
p_line = np.array([p(point) for point in points])
plt.plot(y, 50*w_line, 'k', linewidth=2)  # magnify w
plt.plot(y, p_line, 'b--', linewidth=2)
plt.grid(True)
plt.xlabel('$y$')
plt.legend(['Deflection ($\\times 50$)', 'Load'], loc='upper left')
plt.savefig('poisson_membrane/curves.pdf')
plt.savefig('poisson_membrane/curves.png')

# Hold plots
interactive()
plt.show()

- ```domain = Circle(Point(0, 0), 1)```
> NameError: name 'Circle' is not defined

- ```domain = mshr.Circle(Point(0, 0), 1)```
> AttributeError: module 'mshr' has no attribute 'Circle'

- ```mesh = UnitDiscMesh(object, 64, 1, gdim)```
> TypeError: dolfin.cpp.generation.UnitDiscMesh: No constructor defined!
